# Extracting gait features
Here we use the preprocessed accelerometer data as input, create time windows and extract relevant features for the subsequent task of gait detection. 

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pandas as pd
import tsdf

from dbpd.feature_extraction import *
from dbpd.windowing import *
from dbpd import DataColumns

## Constants

In [2]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'

input_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
output_path = os.path.join(path_to_data, '3.extracted_features', 'gait')

In [3]:
# tsdf files
sensor = 'accelerometer'

meta_filename = f'{sensor}_meta.json'
values_filename = f'{sensor}_samples.bin'
time_filename = f'{sensor}_time.bin'

# windowing and feature engineering
window_type = 'hann'
verbose = 0

sampling_frequency = 100 # Hz
window_length_s = 6
window_step_size_s = 1

# cepstral coefficients
low_frequency = 0 # Hz
high_frequency = int(sampling_frequency / 2) # Hz
filter_length = high_frequency - 1 # here we assume a 1 Hz gait common frequency with equally-spaced harmonics
n_dct_filters = 16 # number of cepstral coefficients
filter_lenth = 16 # length of the filter

d_frequency_bandwidths = {
    'power_below_gait': [0.3, 0.7],
    'power_gait': [0.7, 3.5],
    'power_tremor': [3.5, 8],
    'power_above_tremor': [8, sampling_frequency]
}

l_accelerometer_cols = [DataColumns.ACCELEROMETER_X, DataColumns.ACCELEROMETER_Y, DataColumns.ACCELEROMETER_Z]
l_gravity_cols = [f'grav_{x}' for x in l_accelerometer_cols]
l_window_level_cols = ['id', 'window_nr', 'window_start', 'window_end']
l_data_point_level_cols = l_accelerometer_cols + l_gravity_cols

# store data
d_channels_values = {
    'grav_accelerometer_x_mean': 'g',
    'grav_accelerometer_y_mean': 'g',
    'grav_accelerometer_z_mean': 'g',
    'grav_accelerometer_x_std': 'g',
    'grav_accelerometer_y_std': 'g',
    'grav_accelerometer_z_std': 'g',
    'accelerometer_x_power_below_gait': 'X',
    'accelerometer_y_power_below_gait': 'X',
    'accelerometer_z_power_below_gait': 'X',
    'accelerometer_x_power_gait': 'X',
    'accelerometer_y_power_gait': 'X',
    'accelerometer_z_power_gait': 'X',
    'accelerometer_x_power_tremor': 'X',
    'accelerometer_y_power_tremor': 'X',
    'accelerometer_z_power_tremor': 'X',
    'accelerometer_x_power_above_tremor': 'X',
    'accelerometer_y_power_above_tremor': 'X',
    'accelerometer_z_power_above_tremor': 'X',
    'accelerometer_x_dominant_frequency': 'Hz',
    'accelerometer_y_dominant_frequency': 'Hz',
    'accelerometer_z_dominant_frequency': 'Hz',
    'std_norm_acc': 'X',
    'cc_1_acc': 'X',
    'cc_2_acc': 'X',
    'cc_3_acc': 'X',
    'cc_4_acc': 'X',
    'cc_5_acc': 'X',
    'cc_6_acc': 'X',
    'cc_7_acc': 'X',
    'cc_8_acc': 'X',
    'cc_9_acc': 'X',
    'cc_10_acc': 'X',
    'cc_11_acc': 'X',
    'cc_12_acc': 'X',
    'cc_13_acc': 'X',
    'cc_14_acc': 'X',
    'cc_15_acc': 'X',
    'cc_16_acc': 'X'
}

## Load data

In [4]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(input_path, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.head(2)

,time,accelerometer_x,grav_accelerometer_x,accelerometer_y,grav_accelerometer_y,accelerometer_z,grav_accelerometer_z
0,0.00,-0.537320,-4.240030e-09,0.560195,4.420535e-09,0.267026,2.107122e-09
1,0.01,-0.496498,-3.783814e-08,0.580556,3.994549e-08,0.235247,1.871333e-08


## Preprocess data

In [5]:
df_windowed = tabulate_windows(
    df=df,
    time_column_name='time',
    data_point_level_cols=l_data_point_level_cols,
    window_length_s=window_length_s,
    window_step_size_s=window_step_size_s,
    sampling_frequency=sampling_frequency
    )

print(df_windowed.shape)

(725, 9)


In [6]:
# extract mean and std of gravity acceleration
for col in l_gravity_cols:
    for stat in ['mean', 'std']:
        df_windowed[f'{col}_{stat}'] = generate_statistics(
            sensor_col=df_windowed[col],
            statistic=stat
            )

In [7]:
# extract standard deviation of the Euclidean norm of the three axes
df_windowed['std_norm_acc'] = generate_std_norm(
    df=df_windowed,
    cols=l_accelerometer_cols
    )

In [8]:
for col in l_accelerometer_cols:

    # fast fourier transforms
    df_windowed[f'{col}_freqs'], df_windowed[f'{col}_fft'] = signal_to_ffts(
        sensor_col=df_windowed[col],
        window_type=window_type,
        sampling_frequency=sampling_frequency
        )

    # compute power in distinct frequency bandwidths
    for bandwidth in d_frequency_bandwidths.keys():
        df_windowed[col+'_'+bandwidth] = df_windowed.apply(lambda x: compute_power_in_bandwidth(
            sensor_col=x[col],
            fmin=d_frequency_bandwidths[bandwidth][0],
            fmax=d_frequency_bandwidths[bandwidth][1],
            sampling_frequency=sampling_frequency,
            window_type=window_type,
            ), axis=1
        )

    # extract dominant frequency
    df_windowed[col+'_dominant_frequency'] = df_windowed.apply(lambda x: get_dominant_frequency(
        signal_ffts=x[col+'_fft'], 
        signal_freqs=x[col+'_freqs'],
        fmin=d_frequency_bandwidths[bandwidth][0],
        fmax=d_frequency_bandwidths[bandwidth][1]
        ), axis=1
    )

In [9]:
for bandwidth in d_frequency_bandwidths.keys():
    df_windowed['total_acc_'+bandwidth] = df_windowed.apply(lambda x: sum(x[y+'_'+bandwidth] for y in l_accelerometer_cols), axis=1)

df_windowed['total_accel_power'] = compute_power(
    df=df_windowed,
    fft_cols=[f'{col}_fft' for col in l_accelerometer_cols])

cc_cols = generate_cepstral_coefficients(
    total_power_col=df_windowed['total_accel_power'],
    window_length_s=window_length_s,
    sampling_frequency=sampling_frequency,
    low_frequency=low_frequency,
    high_frequency=high_frequency,
    filter_length=filter_length,
    n_dct_filters=n_dct_filters
    )

df_windowed = pd.concat([df_windowed, cc_cols], axis=1)      

In [10]:
df_windowed = df_windowed.rename(columns={f'cc_{cc_nr}': f'cc_{cc_nr}_acc' for cc_nr in range(1,17)}).rename(columns={'window_start': 'time'})

df_windowed = df_windowed.drop(columns=[f'{col}{x}' for x in ['', '_freqs', '_fft', '_fft_power'] for col in l_accelerometer_cols] + ['total_accel_power', 'window_nr', 'window_end'] + l_gravity_cols + l_accelerometer_cols)

## Store data

In [11]:
from dateutil import parser
import datetime

end_iso8601 = (parser.parse(metadata_samples.start_iso8601) + datetime.timedelta(seconds=int(df_windowed['time'][-1:].values[0] + window_length_s))).strftime('%d-%b-%Y %H:%M:%S') + ' UTC'

metadata_samples.__setattr__('end_iso8601', end_iso8601)
metadata_samples.__setattr__('file_name', 'gait_values.bin')
metadata_samples.__setattr__('file_dir_path', output_path)
metadata_time.__setattr__('end_iso8601', end_iso8601)
metadata_time.__setattr__('file_name', 'gait_time.bin')
metadata_time.__setattr__('file_dir_path', output_path)

metadata_samples.__setattr__('channels', list(d_channels_values.keys()))
metadata_samples.__setattr__('units', list(d_channels_values.values()))

metadata_time.__setattr__('channels', ['time'])
metadata_time.__setattr__('units', ['relative_time_ms'])
metadata_time.__setattr__('data_type', np.int64)

In [12]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(output_path, df_windowed, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], 'gait_meta.json')